# Write time-scaled PageRank to all nodes

In [28]:
import json
import time
from functools import reduce

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm.autonotebook import tqdm

from py2neo import Graph, Node, Relationship

In [29]:
graph = Graph("bolt://dev_neo4j:7687", auth=('neo4j','myneo'))

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

def run_query(query, graph, print_query=False, run_query=True, 
              print_only=False, to_df=False, verbose=True):
    df = 1
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        if to_df:
            df = graph.run(query).to_data_frame()
        else:
            graph.run(query)
    end_time = time.time()
    minutes_elapsed = (end_time-start_time)/60
    if verbose:
        print("Query completed in {:.2f} minutes.".format(minutes_elapsed))
    return df

Connected to graph database with 370,269,897 nodes and 220,155,390 relationships!


In [47]:
min_year, max_year = 1900, 1900
for y in tqdm(range(min_year, max_year+1)):
    
    # Write PageRank scores considering all nodes from year <= y
    query = """
    CALL algo.pageRank('
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(z:Year)
    WHERE z.value <= {}
    RETURN id(q) as id
    ','
    MATCH (q1:Quanta)-[:CITES]->(q2:Quanta)
    RETURN id(q1) AS source, id(q2) AS target
    ',{{graph:'cypher', write:true, writeProperty:"temporary"}});
    """.format(y,y)
    run_query(query, graph)
    
    # Calculate reference set statistics for all years <= y
    query = """
    MATCH (y:Year)
    WHERE y.value<=1900
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(z:Year)
    WHERE (z.value<=y.value+1) AND (z.value>=y.value-1)
    RETURN y.value as year, avg(q.temporary) as avg, stdev(q.temporary) AS std
    ORDER BY y.value DESC;
    """.format(y)
    df = run_query(query, graph, to_df=True)
    
    # Normalize score for each Quanta from year <= y
    df['avg_str'] = df.apply(lambda row: 'WHEN {:.0f} THEN {}'.format(row['year'], row['avg']), axis=1)
    df['std_str'] = df.apply(lambda row: 'WHEN {:.0f} THEN {}'.format(row['year'], row['std']), axis=1)
    avg_case_str = 'CASE q.year ' + reduce(lambda a,b:'{} {}'.format(a,b), df['avg_str']) + ' END'
    std_case_str = 'CASE q.year ' + reduce(lambda a,b:'{} {}'.format(a,b), df['std_str']) + ' END'
    query = """
    CALL apoc.periodic.iterate('
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(z:Year) 
    WHERE z.value <= {}
    RETURN q
    ','
    SET q.temporary = ABS(q.temporary-{})/{}
    ',{{batchSize:10000, parallel:true}});
    """.format(year, avg_case_str, std_case_str)
    run_query(query, graph, print_only=False)
    
    # Write scores to METRICS_IN relationship
    query = """
    CALL apoc.periodic.iterate('
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(z:Year) 
    WHERE z.value <= {}
    RETURN q
    ','
    MATCH (y:Year)
    WHERE y.value = {}
    MERGE (q)-[m:METRICS_IN]->(y)
    SET m.timeScaledPageRank = q.temporary
    ',{batchSize:10000, parallel:true});
    """


Query completed in 0.68 minutes.
Query completed in 0.02 minutes.

    CALL apoc.periodic.iterate('
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(z:Year) 
    WHERE z.value <= 2018
    RETURN q
    ','
    SET q.temporary = ABS(q.temporary-CASE q.year WHEN 1900 THEN 0.15015496810061815 WHEN 1899 THEN 0.1501723712509411 WHEN 1898 THEN 0.1501949793298881 WHEN 1897 THEN 0.15013476057412878 WHEN 1896 THEN 0.15017029551361163 WHEN 1895 THEN 0.15024061653390883 WHEN 1894 THEN 0.15037512690514931 WHEN 1893 THEN 0.1503085383149771 WHEN 1892 THEN 0.15026635515077102 WHEN 1891 THEN 0.1501403398235669 WHEN 1890 THEN 0.15012390671099837 WHEN 1889 THEN 0.1500732291031012 WHEN 1888 THEN 0.15004793233273134 WHEN 1887 THEN 0.15004876960540556 WHEN 1886 THEN 0.15001682058114277 WHEN 1885 THEN 0.15003552521734873 WHEN 1884 THEN 0.15001788469702093 WHEN 1883 THEN 0.150068202662041 WHEN 1882 THEN 0.1500522027534668 WHEN 1881 THEN 0.15005606860381096 WHEN 1880 THEN 0.1500110927444812 WHEN 1879 THEN 0.150011665142

In [42]:
avg_case_str

'CASE p.year WHEN 1899 THEN 0.1501723712509411 WHEN 1898 THEN 0.1501949793298881 WHEN 1897 THEN 0.15013476057412878 WHEN 1896 THEN 0.15017029551361163 WHEN 1895 THEN 0.15024061653390883 WHEN 1894 THEN 0.15037512690514931 WHEN 1893 THEN 0.1503085383149771 WHEN 1892 THEN 0.15026635515077102 WHEN 1891 THEN 0.1501403398235669 WHEN 1890 THEN 0.15012390671099837 WHEN 1889 THEN 0.1500732291031012 WHEN 1888 THEN 0.15004793233273134 WHEN 1887 THEN 0.15004876960540556 WHEN 1886 THEN 0.15001682058114277 WHEN 1885 THEN 0.15003552521734873 WHEN 1884 THEN 0.15001788469702093 WHEN 1883 THEN 0.150068202662041 WHEN 1882 THEN 0.1500522027534668 WHEN 1881 THEN 0.15005606860381096 WHEN 1880 THEN 0.1500110927444812 WHEN 1879 THEN 0.15001166514227507 WHEN 1878 THEN 0.1500127372632435 WHEN 1877 THEN 0.15000000000000002 WHEN 1876 THEN 0.15001522206364196 WHEN 1875 THEN 0.15001585820958538 WHEN 1874 THEN 0.15001592157905405 WHEN 1873 THEN 0.15000000000000002 WHEN 1872 THEN 0.15008306189255124 WHEN 1871 THEN 0.

In [ ]:
MATCH (y:Year) WHERE y.value=2019
CALL algo.pageRank.stream('
MATCH (q:Quanta)-[:PUBLISHED_IN]->(z:Year)
WHERE z.value >= 1900 AND z.value <= y.value
RETURN id(q) as id
','
MATCH (q1:Quanta)-[:CITES]->(q2:Quanta)
RETURN id(q1) AS source, id(q2) AS target
',{graph:'cypher'})
yield nodeId, score
MATCH (q:Quanta) WHERE id(q)=nodeId
MERGE (q)-[m:METRICS_IN]->(y)
SET m.tspr = score

In [ ]:
MATCH (q:Quanta)-[:PUBLISHED_IN]->(y:Year)
WHERE id(q)=57383
MATCH (z:Year) WHERE z.value >= y.value
MERGE (q)-[m:METRICS_IN]->(z)
SET m.cites = size((z)<-[:PUBLISHED_IN]-(:Quanta)-[:CITES]->(q))

# OLD

In [ ]:
import pandas as pd
import json
from py2neo import Graph, Node, Relationship
import numpy as np
import time
from functools import reduce
from tqdm import tqdm

In [ ]:
graph = Graph("bolt://neo4j-magtwo:7687", auth=('neo4j','myneo'))

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

In [20]:
start_year, end_year, step = 2018, 2020, 1
min_year = 1950

In [21]:
# Write time-series of PageRank values to Quanta
print("Writing PageRank for {} to {}...".format(start_year, end_year))

start_time = time.time()
for year in tqdm(range(start_year, end_year+1, step)):
    year_start_time = time.time()
    
    where_clause = 'WHERE p.year < {} AND p.year > {} AND p.venue="Nature"'.format(year+1, min_year-1)
    
    # [1] Write standard PageRank values for current-year graph
    print("\t{}: Writing standard PageRank values...".format(year), end=" ")
    query_start_time = time.time()
    query = """
    CALL algo.pageRank(
    'MATCH (p:Quanta) {} RETURN id(p) as id',
    'MATCH (p1:Quanta)-[:CITES]->(p2:Quanta) RETURN id(p1) as source, id(p2) as target',
    {{graph:'cypher', iterations:35, write:true, writeProperty:"tspr{}"}});
    """.format(where_clause, year)
    print(query)
#     graph.run(query)
    print("Done ({:.2f} min).".format((time.time()-query_start_time)/60))
      
    # [2] Collect reference-set normalization values for all years up to the current year
    print("\t{}: Calculating reference-set statistics FOR SAME YEAR...".format(year), end=" ")
    query_start_time = time.time()    
    query = """
    MATCH (p:Quanta)
    {}
    RETURN 
        p.year as year,
        AVG(p.tspr{}) as avgpr, 
        stDev(p.tspr{}) as stdevpr
    ORDER BY year DESC
    """.format(where_clause, year, year)
    print(query)
#     df = graph.run(query).to_data_frame()
    avg_pagerank = df['avgpr'].iloc[0]
    stdev_pagerank = df['stdevpr'].iloc[0]
    print("Finished ({:.2f} min).".format((time.time()-query_start_time)/60))  
    
    # [3] Normalize current-year PageRank values using calculated reference set from current year
    print("\t{}: Normalizing to time-scaled PageRank values...".format(year), end=" ")
    
    df['avg_str'] = df.apply(lambda row: 'WHEN {:.0f} THEN {}'.format(row['year'], row['avgpr']), axis=1)
    df['std_str'] = df.apply(lambda row: 'WHEN {:.0f} THEN {}'.format(row['year'], row['stdevpr']), axis=1)
    avg_case_str = 'CASE p.year ' + reduce(lambda a,b:'{} {}'.format(a,b), df['avg_str']) + ' END'
    std_case_str = 'CASE p.year ' + reduce(lambda a,b:'{} {}'.format(a,b), df['std_str']) + ' END'
    
    query_start_time = time.time()
    query = """
    CALL apoc.periodic.iterate(
    'MATCH (p:Quanta) {} RETURN p',
    'SET p.tsprn{} = ABS(p.tspr{}-{})/{}',
    {{batchSize:10000, parallel:true}})
    """.format(where_clause, year, year, avg_case_str, std_case_str)
    print(query)
#     graph.run(query)
    print("Finished writing ({:.2f} min).".format((time.time()-query_start_time)/60))        
    
    print("\t{}: Wrote ts-PageRank to \"tspr{}\" ({:.2f} min)".format(year, year, (time.time()-year_start_time)/60))
    
print("Finished everything ({:.2f} min).".format((time.time()-start_time)/60))

  0%|          | 0/3 [00:00<?, ?it/s]

Writing PageRank for 2018 to 2020...
	2018: Writing standard PageRank values... 
    CALL algo.pageRank(
    'MATCH (p:Quanta) WHERE p.year < 2019 AND p.year > 1949 AND p.venue="Nature" RETURN id(p) as id',
    'MATCH (p1:Quanta)-[:CITES]->(p2:Quanta) RETURN id(p1) as source, id(p2) as target',
    {graph:'cypher', iterations:35, write:true, writeProperty:"tspr2018"});
    
Done (0.00 min).
	2018: Calculating reference-set statistics FOR SAME YEAR... 
    MATCH (p:Quanta)
    WHERE p.year < 2019 AND p.year > 1949 AND p.venue="Nature"
    RETURN 
        p.year as year,
        AVG(p.tspr2018) as avgpr, 
        stDev(p.tspr2018) as stdevpr
    ORDER BY year DESC
    


NameError: name 'df' is not defined